# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
wthr = "C:/Users/school/Desktop/api_hw/python-api-challenge/WeatherPy/cities.csv"

wthr_df = pd.read_csv(wthr)

wthr_df.head(-1)


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,hobart,AU,58.26,58,40,7.00,-42.8794,147.3294,1627439606
1,heinola,FI,64.49,58,100,4.47,61.2056,26.0381,1627439606
2,pervomayskoye,RU,69.69,56,100,12.19,43.9190,46.7102,1627439607
3,carlsbad,US,88.27,83,40,1.99,33.1581,-117.3506,1627439405
4,swan river,CA,78.69,57,68,3.44,52.1058,-101.2676,1627439415
...,...,...,...,...,...,...,...,...,...
561,mayumba,GA,72.70,88,99,2.66,-3.4320,10.6554,1627439696
562,constitucion,CL,44.65,78,0,7.23,-35.3333,-72.4167,1627439752
563,yingcheng,CN,84.25,73,62,18.54,30.9500,113.5500,1627439753
564,edson,CA,73.54,43,1,5.75,53.5835,-116.4356,1627439511


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#Configure gmaps.
gmaps.configure(api_key = g_key)

#get loc and humidity
city_loc = wthr_df[['Latitude', 'Longitude']].astype(float)
humidity = wthr_df['Humidity'].astype(float)


In [9]:
#plot
fig = gmaps.figure(map_type='HYBRID')

heat_layer = gmaps.heatmap_layer(city_loc, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#add layer
fig.add_layer(heat_layer)
#display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
#set my own conditions
best_wthr = wthr_df.loc[(wthr_df['Temperature'] >= 68) & (wthr_df['Temperature'] <= 69) & 
                                  (wthr_df['Windspeed'] <= 68) & (wthr_df['Humidity'] <= 69)]
best_wthr

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
117,thompson,CA,68.16,60,75,4.61,55.7435,-97.8558,1627439634
198,marovoay,MG,68.14,64,93,5.82,-16.1000,46.6333,1627439656
234,babanka,UA,68.74,58,100,7.52,48.7097,30.4483,1627439664
274,vasterhaninge,SE,68.45,44,99,4.27,59.1167,18.1000,1627439675
333,ozinki,RU,68.52,57,8,10.45,51.1786,49.6769,1627439691
341,derzhavinsk,KZ,68.47,34,6,10.96,51.1000,66.3167,1627439693
411,caohai,CN,68.97,69,59,4.85,26.8661,104.2841,1627439712
452,russkaya polyana,UA,68.38,58,46,8.10,49.4170,31.9201,1627439722
464,almendralejo,ES,68.27,60,0,4.00,38.6832,-6.4075,1627439725


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = best_wthr
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,thompson,CA,68.16,60,75,4.61,55.7435,-97.8558,1627439634
1,marovoay,MG,68.14,64,93,5.82,-16.1000,46.6333,1627439656
2,babanka,UA,68.74,58,100,7.52,48.7097,30.4483,1627439664
3,vasterhaninge,SE,68.45,44,99,4.27,59.1167,18.1000,1627439675
4,ozinki,RU,68.52,57,8,10.45,51.1786,49.6769,1627439691
5,derzhavinsk,KZ,68.47,34,6,10.96,51.1000,66.3167,1627439693
6,caohai,CN,68.97,69,59,4.85,26.8661,104.2841,1627439712
7,russkaya polyana,UA,68.38,58,46,8.10,49.4170,31.9201,1627439722
8,almendralejo,ES,68.27,60,0,4.00,38.6832,-6.4075,1627439725


In [19]:
params = {
    "radius": 4000,
    "types": "lodging",
    "key": g_key
}

#for loop
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # reloacte and iterate with params
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make url and pull request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    
    # get results
    results = response['results']
    
    try:
        print(f"You are near hotel {results[0]['name']}.")
        print(f" rating of nearest hotel {results[0]['rating']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Rate'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: thompson.
You are near hotel Burntwood Hotel.
 rating of nearest hotel 2.8.
------------
Retrieving Results for Index 1: marovoay.
Missing field/result... skipping.


C:\Users\school\.conda\envs\pythondata\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\school\.conda\envs\pythondata\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


------------
Retrieving Results for Index 2: babanka.
You are near hotel Отель Колумб.
 rating of nearest hotel 4.3.
------------
Retrieving Results for Index 3: vasterhaninge.
You are near hotel Viamap.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: ozinki.
You are near hotel Korolevskiy Dvor.
 rating of nearest hotel 4.8.
------------
Retrieving Results for Index 5: derzhavinsk.
You are near hotel Ooo.khim Baza.
 rating of nearest hotel 3.5.
------------
Retrieving Results for Index 6: caohai.
You are near hotel Jinhua Hotel.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: russkaya polyana.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: almendralejo.
You are near hotel Hotel España.
 rating of nearest hotel 3.7.
------------


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
print(hotel_df.columns)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# figure
fig

Index(['City', 'Country', 'Temperature', 'Humidity', 'Cloudiness', 'Windspeed',
       'Latitude', 'Longitude', 'Date', 'Hotel Name', 'Hotel Rate'],
      dtype='object')


Figure(layout=FigureLayout(height='420px'))